In [1]:
import torch, detectron2
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

torch:  2.0 ; cuda:  cpu
detectron2: 0.6


In [7]:
import glob, os

In [5]:
import torch
import torch.nn as nn
from torchvision.transforms import CenterCrop
from torch.nn import functional as F

# Data preparation

Collecting images of rods from the [web-page](https://www.istockphoto.com/pl/search/search-by-asset?searchbyasset=true&assettype=image&istockcollection=main%2Cvalue&assetid=186326832)

In [3]:
import requests
URL = "https://www.istockphoto.com/pl/search/search-by-asset?searchbyasset=true&assettype=image&istockcollection=main%2Cvalue&assetid=186326832" # Replace this with the website's URL

getURL = requests.get(URL, headers={"User-Agent": "Chrome/114.0.0.0"})
print(getURL.status_code)

200


In [4]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(getURL.text, 'html.parser')

images = soup.find_all('img')
print(images)

[<img alt="similar content thumbnail" class="bBLoTnS_I_60C1s1J8AL" src="https://media.istockphoto.com/id/186326832/pl/zdj%C4%99cie/sta%C5%82e-aluminium-prob%C3%B3wek.jpg?s=170x170&amp;k=20&amp;c=fjaf04nu47rLqnaLrw6B4jGliv8iCb8e4ZE5c3oYpK8="/>, <img alt="Stos ze stali okrągłej – zdjęcie" class="yGh0CfFS4AMLWjEE9W7v" fetchpriority="high" height="612" src="https://media.istockphoto.com/id/1254271728/pl/zdj%C4%99cie/stos-ze-stali-okr%C4%85g%C5%82ej.jpg?s=612x612&amp;w=0&amp;k=20&amp;c=l3-k5UtLVJhvA5XJN_GHoxh2GJh9WAR8ak-ZlqrlQW0=" width="408"/>, <img alt="Okrągła rura stalowa w fabryce, tło tekstury – zdjęcie" class="yGh0CfFS4AMLWjEE9W7v" height="408" src="https://media.istockphoto.com/id/1254273927/pl/zdj%C4%99cie/okr%C4%85g%C5%82a-rura-stalowa-w-fabryce-t%C5%82o-tekstury.jpg?s=612x612&amp;w=0&amp;k=20&amp;c=FfjpJjr2gQs2HACPZE6MJIKvBGZn5vdk3FiQbcVEG4Y=" width="612"/>, <img alt="stalowe rury przewodowe elektryczne – zdjęcie" class="yGh0CfFS4AMLWjEE9W7v" height="408" src="https://media.istoc

In [66]:
imageSources = []

for image in images:
    imageSources.append(image.get('src'))

print(imageSources)

['https://media.istockphoto.com/id/186326832/pl/zdj%C4%99cie/sta%C5%82e-aluminium-prob%C3%B3wek.jpg?s=170x170&k=20&c=fjaf04nu47rLqnaLrw6B4jGliv8iCb8e4ZE5c3oYpK8=', 'https://media.istockphoto.com/id/1254271728/pl/zdj%C4%99cie/stos-ze-stali-okr%C4%85g%C5%82ej.jpg?s=612x612&w=0&k=20&c=l3-k5UtLVJhvA5XJN_GHoxh2GJh9WAR8ak-ZlqrlQW0=', 'https://media.istockphoto.com/id/1254273927/pl/zdj%C4%99cie/okr%C4%85g%C5%82a-rura-stalowa-w-fabryce-t%C5%82o-tekstury.jpg?s=612x612&w=0&k=20&c=FfjpJjr2gQs2HACPZE6MJIKvBGZn5vdk3FiQbcVEG4Y=', 'https://media.istockphoto.com/id/1160041931/pl/zdj%C4%99cie/stalowe-rury-przewodowe-elektryczne.jpg?s=612x612&w=0&k=20&c=tLZFzM9xsr4SlfNlXpSwYxJj19ElE59ACFArQWmkzsE=', 'https://media.istockphoto.com/id/1386788089/pl/zdj%C4%99cie/stos-czarnych-d%C5%82ugich-%C5%BCelaznych-rur-umieszczonych-na-p%C3%B3%C5%82ce.jpg?s=612x612&w=0&k=20&c=HLGJri3qwSOtAAl5K3d8rdFyk0muGgpTSmzREn0bGfg=', 'https://media.istockphoto.com/id/536289339/pl/zdj%C4%99cie/runda-billet-pr%C4%99ty.jpg?s=612x612

In [67]:
len(imageSources)

61

In [68]:
#go through 100 pages with images of rods
for i in range(2,101):
    URL=f"https://www.istockphoto.com/pl/search/search-by-asset?searchbyasset=true&assettype=image&istockcollection=main%2Cvalue&page={i}&assetid=186326832"
    getURL = requests.get(URL, headers={"User-Agent": "Chrome/114.0.0.0"})
    soup = BeautifulSoup(getURL.text, 'html.parser')
    images = soup.find_all('img')
    for image in images:
        imageSources.append(image.get('src'))

In [71]:
len(imageSources)

6100

In [88]:
imageSources = list(set(imageSources))

In [89]:
len(imageSources)

5992

I will leave first 100 images for train, 40 for validation and 20 for test.

### Save all images

In [112]:
i=0
for image in imageSources:
    webs = requests.get(image)
    if webs.status_code == 200:
        i+=1
        if i < 100:
            folder = 'train/'
        elif i < 160:
            folder='val/'
        elif i <=180:
            folder = 'test/'
        else:
            break
    open('images/' + folder + image.split('/')[-1].split("?")[0], 'wb').write(webs.content)

In [114]:
image_file_names = {'test':[], 'train':[], 'val':[]}
for folder in glob.glob("images" + '/*'):
    for filename in glob.glob(folder + "/*.jpg"):
        folder_name=folder.split('\\')[-1]
        image_file_names[folder_name].append(filename)
    print(f"images in {folder_name}: ", len(image_file_names[folder_name]))


images in test:  20
images in train:  94
images in val:  40


94 instead of 100 maybe because there were some duplicated images.

After cleaning images we have:

In [3]:
image_file_names = {'test':[], 'train':[], 'val':[]}
for folder in glob.glob("images" + '/*'):
    for filename in glob.glob(folder + "/*.jpg"):
        folder_name=folder.split('\\')[-1]
        image_file_names[folder_name].append(filename)
    print(f"images in {folder_name}: ", len(image_file_names[folder_name]))


images in test:  10
images in train:  47
images in val:  40


In [8]:
# define the path to the images and masks dataset
IMAGE_DATASET_PATH = os.path.join("images", "train")
MASK_DATASET_PATH = os.path.join("masks", "train_masks")
# define the test split
TEST_SPLIT = 0.15
# determine the device to be used for training and evaluation
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
# define the number of channels in the input, number of classes,
# and number of levels in the U-Net model
NUM_CHANNELS = 3
NUM_CLASSES = 1
NUM_LEVELS = 3
# initialize learning rate, number of epochs to train for, and the
# batch size
INIT_LR = 0.001
NUM_EPOCHS = 40
BATCH_SIZE = 4
# define the input image dimensions
INPUT_IMAGE_WIDTH = 128
INPUT_IMAGE_HEIGHT = 128
# define threshold to filter weak predictions
THRESHOLD = 0.5
# define the path to the base output directory
BASE_OUTPUT = "output"
# define the path to the output serialized model, model training
# plot, and testing image paths
MODEL_PATH = os.path.join(BASE_OUTPUT, "rod_model.pth")
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])
TEST_PATHS = os.path.sep.join([BASE_OUTPUT, "test_paths.txt"])

### Define custom dataset for segmentation

In [11]:
from torch.utils.data import Dataset
import cv2


class SegmentationDataset(Dataset):
	def __init__(self, imagePaths, maskPaths, transforms):
		# store the image and mask filepaths, and augmentation
		# transforms
		self.imagePaths = imagePaths
		self.maskPaths = maskPaths
		self.transforms = transforms

	def __len__(self):
		# return the number of total samples contained in the dataset
		return len(self.imagePaths)

	def __getitem__(self, idx):
		# grab the image path from the current index
		imagePath = self.imagePaths[idx]
		# load the image from disk, swap its channels from BGR to RGB,
		# and read the associated mask from disk in grayscale mode
		image = cv2.imread(imagePath)
		image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
		mask = cv2.imread(self.maskPaths[idx], 0)
		# check to see if we are applying any transformations
		if self.transforms is not None:
			# apply the transformations to both image and its mask
			image = self.transforms(image)
			mask = self.transforms(mask)
		# return a tuple of the image and its mask
		return (image, mask)

### U-net model architecture

In [12]:
class Block(nn.Module):
	def __init__(self, inChannels, outChannels):
		super().__init__()
		# store the convolution and RELU layers
		self.conv1 = nn.Conv2d(inChannels, outChannels, 3)
		self.relu = nn.ReLU()
		self.conv2 = nn.Conv2d(outChannels, outChannels, 3)

	def forward(self, x):
		# apply CONV => RELU => CONV block to the inputs and return it
		return self.conv2(self.relu(self.conv1(x)))

In [13]:
class Encoder(nn.Module):
	def __init__(self, channels=(3, 16, 32, 64)):
		super().__init__()
		# store the encoder blocks and maxpooling layer
		self.encBlocks = nn.ModuleList(
			[Block(channels[i], channels[i + 1])
			 	for i in range(len(channels) - 1)])
		self.pool = nn.MaxPool2d(2)


	def forward(self, x):
		# initialize an empty list to store the intermediate outputs
		blockOutputs = []
		# loop through the encoder blocks
		for block in self.encBlocks:
			# pass the inputs through the current encoder block, store
			# the outputs, and then apply maxpooling on the output
			x = block(x)
			blockOutputs.append(x)
			x = self.pool(x)
		# return the list containing the intermediate outputs
		return blockOutputs

In [14]:
class Decoder(nn.Module):
	def __init__(self, channels=(64, 32, 16)):
		super().__init__()
		# initialize the number of channels, upsampler blocks, and
		# decoder blocks
		self.channels = channels
		self.upconvs = nn.ModuleList(
			[nn.ConvTranspose2d(channels[i], channels[i + 1], 2, 2)
			 	for i in range(len(channels) - 1)])
		self.dec_blocks = nn.ModuleList(
			[Block(channels[i], channels[i + 1])
			 	for i in range(len(channels) - 1)])

	def forward(self, x, encFeatures):
		# loop through the number of channels
		for i in range(len(self.channels) - 1):
			# pass the inputs through the upsampler blocks
			x = self.upconvs[i](x)
			# crop the current features from the encoder blocks,
			# concatenate them with the current upsampled features,
			# and pass the concatenated output through the current
			# decoder block
			encFeat = self.crop(encFeatures[i], x)
			x = torch.cat([x, encFeat], dim=1)
			x = self.dec_blocks[i](x)
		# return the final decoder output
		return x

	def crop(self, encFeatures, x):
		# grab the dimensions of the inputs, and crop the encoder
		# features to match the dimensions
		(_, _, H, W) = x.shape
		encFeatures = CenterCrop([H, W])(encFeatures)
		# return the cropped features
		return encFeatures

In [16]:
class UNet(nn.Module):
	def __init__(self, encChannels=(3, 16, 32, 64),
		 decChannels=(64, 32, 16),
		 nbClasses=1, retainDim=True,
		 outSize=(INPUT_IMAGE_HEIGHT,  INPUT_IMAGE_WIDTH)):
		super().__init__()
		# initialize the encoder and decoder
		self.encoder = Encoder(encChannels)
		self.decoder = Decoder(decChannels)
		# initialize the regression head and store the class variables
		self.head = nn.Conv2d(decChannels[-1], nbClasses, 1)
		self.retainDim = retainDim
		self.outSize = outSize


	def forward(self, x):
		# grab the features from the encoder
		encFeatures = self.encoder(x)
		# pass the encoder features through decoder making sure that
		# their dimensions are suited for concatenation
		decFeatures = self.decoder(encFeatures[::-1][0],
			encFeatures[::-1][1:])
		# pass the decoder features through the regression head to
		# obtain the segmentation mask
		map = self.head(decFeatures)
		# check to see if we are retaining the original output
		# dimensions and if so, then resize the output to match them
		if self.retainDim:
			map = F.interpolate(map, self.outSize)
		# return the segmentation map
		return map